# Title

## Load libraries

In [ ]:
%pylab inline

In [ ]:
import os
import _iso
import numpy as np
import numpy.ma as ma
from mpl_toolkits.basemap import Basemap, shiftgrid
from matplotlib.mlab import griddata
import matplotlib.colors as colors
from scipy.signal import medfilt2d
import netCDF4
import scipy.interpolate
from pandas import Series, DataFrame
import pandas as pd

import pyroms
import pyroms_toolbox
from bathy_smoother import *

In [ ]:
os.environ["PYTHONPATH"] = "/Users/misumi:/usr/local/lib/python2.7/site-packages"
os.environ["PYROMS"] = "$PYTHONPATH/pyroms"
os.environ["PYROMS_GRIDID_FILE"] = "/Users/misumi/roms_data/gridid.txt"

## Def funcs

In [ ]:
def remap_woa_to_h025(var):
    rmp_file = 'remap_woa_to_h025_l045_v001_bilin.nc'
    #var_out = pyroms.remapping.remap(var[:,1:,1:].copy(),rmp_file,spval=np.nan)
    var_out = pyroms.remapping.remap(var[:,:,:].copy(),rmp_file,spval=np.nan)
    return var_out

In [ ]:
def fill_mask_woa(var):
    a = var[:,:].copy()
    a = a.flatten()

    lat = grd_woa.hgrid.lat_rho[:]
    lon = grd_woa.hgrid.lon_rho[:]

    lat_1d = lat.flatten()
    lon_1d = lon.flatten()

    lat_1d = ma.masked_where(a.mask,lat_1d)
    lon_1d = ma.masked_where(a.mask,lon_1d)

    a = a.compressed()
    lat_1d = lat_1d.compressed()
    lon_1d = lon_1d.compressed()

    pts = np.zeros((lat_1d.size,2))

    pts[:,0] = lon_1d[:]
    pts[:,1] = lat_1d[:]

    b = scipy.interpolate.griddata(pts,a,(lon,lat),method='nearest')
    
    return b

## Read grids

In [ ]:
grd_woa = pyroms.grid.get_ROMS_grid('woa_an')
grd_h025 = pyroms.grid.get_ROMS_grid('h025_l045_v001')
grd_h025_z = pyroms.grid.get_ROMS_grid('h025_l045_v001_woa_z')

## For write

In [ ]:
dst_nc00 = netCDF4.Dataset('../data/h025_l045_v001.clm-ecosys-woa.161206.nc','w')
dst_nc00.createDimension('ocean_time',12)
dst_nc00.createDimension('s_rho',nz)
dst_nc00.createDimension('eta_rho',ny)
dst_nc00.createDimension('xi_rho',nx)

fv = -999.0
no3_o = dst_nc00.createVariable('NO3',dtype('float').char,('ocean_time','s_rho','eta_rho','xi_rho'),fill_value=fv)

In [ ]:
no3_o[:] = nudge[:]

In [ ]:
dst_nc00.close()

### Copy dimes and vars from an exisiting file

In [ ]:
src_nc01 = netCDF4.Dataset('../data/h025_l045_v001.ini.161108.nc','r')
dst_nc00 = netCDF4.Dataset('../data/test.nc','w')
vnames = src_nc01.variables.keys()
dnames = src_nc01.dimensions.keys()

for d in dnames:
    x = src_nc01.dimensions.get(d)
    dst_nc00.createDimension(x.name,x.size)
    
fv = -999.0
for v in vnames:
    x = src_nc01.variables[v]
    y = dst_nc00.createVariable(x.name,np.dtype('float').char,(x.dimensions),fill_value=fv)

dst_nc00.close()

## For read

In [ ]:
src_nc00 = netCDF4.Dataset('../data/woa2013v2/woa13_all_no3.nc','r')
var = src_nc00.variables['no3'][:,::-1,:,:]
src_nc00.close()

## Basemap

In [ ]:
fig0 = plt.figure(figsize=(20,15))

vmx = 30.
vmn = 0.

ax0 = fig0.add_subplot(221)
m0 = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
            llcrnrlon=0,urcrnrlon=360,resolution='c')
x,y = m0(grd_woa.hgrid.lon_rho,grd_woa.hgrid.lat_rho)
im0 = m0.pcolor(x,y,var[0,-1,:,:],vmin=vmn,vmax=vmx)
m0.drawcoastlines()
cb0 = m0.colorbar(im0,'bottom',size='5%',pad='2%')

ax1 = fig0.add_subplot(222)
m1 = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
            llcrnrlon=0,urcrnrlon=360,resolution='c')
im1 = m1.pcolor(x,y,var[3,-1,:,:],vmin=vmn,vmax=vmx)
m1.drawcoastlines()
cb1 = m1.colorbar(im1,'bottom',size='5%',pad='2%')

ax2 = fig0.add_subplot(223)
m2 = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
            llcrnrlon=0,urcrnrlon=360,resolution='c')
im2 = m2.pcolor(x,y,var[6,-1,:,:],vmin=vmn,vmax=vmx)
m2.drawcoastlines()
cb2 = m2.colorbar(im2,'bottom',size='5%',pad='2%')

ax3 = fig0.add_subplot(224)
m3 = Basemap(projection='cyl',llcrnrlat=-90,urcrnrlat=90,\
            llcrnrlon=0,urcrnrlon=360,resolution='c')
im3 = m2.pcolor(x,y,var[9,-1,:,:],vmin=vmn,vmax=vmx)
m3.drawcoastlines()
cb3 = m3.colorbar(im2,'bottom',size='5%',pad='2%')

plt.tight_layout()